## Setup

In [1]:
!pip install -qU transformers==4.48.3 datasets==3.2.0 optimum==1.24.0
#!pip install -qU openai==1.61.0 wandb
!pip install -qU json-repair==0.29.1
!pip install -qU faker==35.2.0
#!pip install -qU vllm==0.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 88.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 11.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory && pip install -e .

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 357, done.
remote: Counting objects: 100% (357/357), done.
remote: Compressing objects: 100% (276/276), done.
remote: Total 357 (delta 77), reused 292 (delta 66), pack-reused 0 (from 0)
Receiving objects: 100% (357/357), 9.64 MiB | 35.40 MiB/s, done.
Resolving deltas: 100% (77/77), done.
Obtaining file:///kaggle/working/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 47.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_from_disk, DatasetDict, concatenate_datasets
#path = f"/kaggle/working/multi-lexsum/multi_lexsum"
path = f"/kaggle/input/multi-lexsum/multi_lexsum"
ds   = load_from_disk(path)

In [ ]:
# from google.colab import userdata
import wandb

wandb.login(key='bb588365ecjghjlkflgfdgfdgfdgdf')
hf_token = 'hf_NKNJDwtXMdPgfdgdshrrrfddfhdf'
!huggingface-cli login --token {hf_token}

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: medoahmed0120 (medoahmed0120-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `legal` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `legal`


## Imports

In [5]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime

import json_repair

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

data_dir = "/kaggle/working/"
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"

device = "cuda"
torch_dtype = None

def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None

## Format Finetuning Datasets

In [6]:
#sft_data_path = join(data_dir, "datasets", "sft.jsonl")

system_message = "\n".join([
    "You are a professional NLP data parser.",
    "Follow the provided `Task` by the user to generate the `Output JSON`.",
    "Do not generate any introduction or conclusion."
])

def prepare_data(ds):
    llm_finetuning_data = []
    
    for rec in ds:
        combined_source = ''
        if len(rec['sources']) > 3:
            combined_source = '\n'.join(rec['sources'][:3])
        else:
            combined_source = '\n'.join(rec['sources'])
        
        llm_finetuning_data.append({
            "system": system_message,
            "instruction": "\n".join([
                "# Document:",
                combined_source,
                "",
                "# Task: summarize",
                "# Output JSON:",
                "```json"
            ]),
            "input": "",
            "output": "\n".join([
                "```json",
                # Fix the key typo
                json.dumps({
                    'Summary_long': rec["summary/long"],
                    'Summary_short': rec["summary/short"]  # Fixed key
                }),
                "```"
            ]),
            "history": []
        })
    return llm_finetuning_data

train_data=prepare_data(ds['train'])
val_data=prepare_data(ds['validation'])
test_data=prepare_data(ds['test'])

#random.Random(101).shuffle(llm_finetunning_data)

In [8]:
train_data[0]

{'system': 'You are a professional NLP data parser.\nFollow the provided `Task` by the user to generate the `Output JSON`.\nDo not generate any introduction or conclusion.',
 'instruction': '# Document:\nCase 1:05-cv-00530-D Document 1-1 Filed 09/19/2005 Page 1 of 6\n\nIN\n\nTHE\n\nUNITED\n\nSTATES\n\nDISTRICT\n\nFILD COUR T\n\nP19\n\n.05\n\nNl\n\nel\n\n.s\n\nFOR THE SOUTHERN DISTRICT OF ALABAMA\n\nSOUTHERN DIVISION\n\nEQUAL EMPLOYMENT OPPORTUNITY ]\n\nCOMMISSION, ]\n\n] Plaintiff, ] Civil Action No. OSS- 0\'53a -~\n\nv.\n\n]\n\n]\nCOMPLAINT\n\n] HOUSE OF PHILADELPHIA CENTER, INC . ]\n\nJURY TRIAL DEMAND\n\nDefendant .\n\n]\n]\n] ]\n\nNATURE OF THE ACTION This is an action under Title VII of the Civil Rights Act of 1964 and Title I of the Civil Rights Act of 1991 to correct unlawful employment practices on the basis of sex and to provide appropriate relief to Sharonda Griffin who was adversely affected by such practices . The Commission alleges that the Defendant discriminated against 

In [9]:

os.makedirs(join(data_dir, "datasets", "llamafactory-finetune-data"), exist_ok=True)

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "train.json"), "w") as dest:
    json.dump(train_data, dest, ensure_ascii=False, default=str)

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "val.json"), "w", encoding="utf8") as dest:
    json.dump(val_data, dest, ensure_ascii=False, default=str)

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "test.json"), "w", encoding="utf8") as dest:
    json.dump(test_data, dest, ensure_ascii=False, default=str)

## Finetuning

In [ ]:
# # # Configure LLaMA-Factory for the new datasets

# update /kaggle/working/LLaMA-Factory/data/dataset_info.json and append
# ```
#    "summarizer_finetune_train": {
#         "file_name": "/kaggle/working/datasets/llamafactory-finetune-data/train.json",
#         "columns": {
#             "prompt": "instruction",
#             "query": "input",
#             "response": "output",
#             "system": "system",
#             "history": "history"
#         }
#     },
#     "summarizer_finetune_val": {
#         "file_name": "/kaggle/working/datasets/llamafactory-finetune-data/val.json",
#         "columns": {
#             "prompt": "instruction",
#             "query": "input",
#             "response": "output",
#             "system": "system",
#             "history": "history"
#         }
#     }
# ```

# # https://wandb.ai/mr-bakrianoo/llamafactory/runs/apwbkni9
# # https://wandb.ai/mr-bakrianoo/llamafactory/runs/c5tf0q90

In [10]:
import json
from os import path

# Path to your dataset_info.json
info_path = "/kaggle/working/LLaMA-Factory/data/dataset_info.json"



# 1. Load existing JSON
with open('/kaggle/input/dataset-info-json/dataset_info.json', "r", encoding="utf-8") as f:
    info = json.load(f)



# 3. Write it back (pretty-printed)
with open(info_path, "w", encoding="utf-8") as f:
    json.dump(info, f, ensure_ascii=False, indent=2)

print(f"Updated {path.basename(info_path)} with summarizer entries.")


Updated dataset_info.json with summarizer entries.


In [11]:
%%writefile /kaggle/working/LLaMA-Factory/examples/train_lora/summarizer_finetune.yaml

### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: summarizer_finetune_train
eval_dataset: summarizer_finetune_val
template: qwen
cutoff_len: 3500
# max_samples: 50
overwrite_cache: true
preprocessing_num_workers: 16

### output
# resume_from_checkpoint: /gdrive/MyDrive/youtube-resources/llm-finetuning/models/checkpoint-1500
output_dir: /kaggle/working/llm-finetuning/models/
logging_steps: 10
save_steps: 150
plot_loss: true
# overwrite_output_dir: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 2.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000
#error rec
resume_from_checkpoint: false  # Explicitly disable resume
overwrite_output_dir: true     # Clean previous checkpoints
### eval
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 100

report_to: wandb
run_name: legalx-finetune-llamafactory

push_to_hub: true
export_hub_model_id: "masry1/legal_summarization"
hub_private_repo: true
hub_strategy: checkpoint


Writing /kaggle/working/LLaMA-Factory/examples/train_lora/summarizer_finetune.yaml


In [12]:
!cd LLaMA-Factory/ && llamafactory-cli train /kaggle/working/LLaMA-Factory/examples/train_lora/summarizer_finetune.yaml

2025-05-11 14:14:33.932953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746972874.181201     286 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746972874.250122     286 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[INFO|2025-05-11 14:14:53] llamafactory.cli:143 >> Initializing 4 distributed tasks at: 127.0.0.1:44839
W0511 14:14:54.967000 359 torch/distributed/run.py:793] 
W0511 14:14:54.967000 359 torch/distributed/run.py:793] *****************************************
W0511 14:14:54.967000 359 torch/distributed/run.py:793] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, p

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="cuda:0",
    torch_dtype = torch_dtype
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

2025-05-11 15:18:07.497632: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746976687.517976     112 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746976687.523939     112 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [15]:
finetuned_model_id = "/kaggle/working/llm-finetuning/models/checkpoint-396"
model.load_adapter(finetuned_model_id)

In [16]:
# Load base model for comparison
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="cuda:1",
    torch_dtype=torch_dtype
)
base_tokenizer = AutoTokenizer.from_pretrained(base_model_id)

In [20]:
def prepare_data_test(ds):
    
    prompts=[]
    summary=[]
    for rec in ds:
        
        combined_source = ''
        if len(rec['sources']) > 3:
            combined_source = '\n'.join(rec['sources'][:3])
        else:
            combined_source = '\n'.join(rec['sources'])
        prompt = [
            {
                "role": "system",
                "content": "\n".join([
                    "You are a professional NLP data parser.",
                    "Follow the provided `Task` by the user to generate the `Output JSON`.",
                    "Do not generate any introduction or conclusion."
                ])
            },
            {
                "role": "user",
                "content":  "\n".join([
                    
                     "## Document:",
                    combined_source,
        
                    "# Task: summarize",
                    "# Output JSON:",
                    "```json"
                ])
            }
        ]
        prompts.append(prompt)
        summary.append({
                    'Summary_long': rec["summary/long"],
                    'Summary_short': rec["summary/short"]  # Fixed key
                })
        

    return prompts , summary
        

test_sources, summarys=prepare_data_test(ds['test'])

In [42]:
def generate_resp(messages,model,device):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
                model_inputs.input_ids,
                max_new_tokens=1024,
                do_sample=True,
                temperature=0.8,
                top_p=0.95,
                repetition_penalty=1.1,
                num_return_sequences=1
            )

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

#response = generate_resp(test_sources[0])

In [45]:
print(f'real_summary: {summarys[1]}')

real_summary: {'Summary_long': "On August 28, 2013, an indigent detainee in the Montgomery Municipal Jail filed this lawsuit in the Circuit Court of Montgomery County, Alabama. The plaintiff sued the City of Montgomery and the Honorable Milton J. Westry under 42 U.S.C. § 1983. The petitioner, represented by the Southern Poverty Law Center, requested the court quash the Municipal Court order requiring the petitioner to serve an imprisonment term of 54 days. Petitioner claimed that the Municipal Court order violated  Sixth Amendment, due process, and equal protection clause. The plaintiff was an indigent woman who accumulated $2,714.00 in fines and fees on traffic tickets that she received in 2008 and 2009. On August 20, 2013, the plaintiff was arrested and brought to the Montgomery Municipal Jail. The next day, the plaintiff appeared before Defendant Judge Westry, who told the plaintiff that she must pay $1,554.00 immediately or serve 31 days in jail. When the plaintiff informed Judge W

In [62]:
response_base=generate_resp(test_sources[1],base_model, 'cuda:1')
print(f'base_model_summary: {response_base}')


base_model_summary: {
  "task": "summarize",
  "output": {
    "title": "Summary of Harriet Delores Cleveland's Amended Complaint",
    "summary": [
      "Harriet Delores Cleveland appeals from her incarceration due to inability to pay traffic fines, claiming violations of the U.S. and Alabama constitutions and Alabama law.",
      "She alleges the Municipal Court lacked procedural safeguards, particularly concerning her indigency and ability to pay fines.",
      "Key points include: Municipal Court's denial of plea and payment plans, lack of investigation into Cleveland's indigency, failure to assign counsel, and mistreatment due to her inability to pay fines.",
      "Claims include deprivation of due process, equal protection, right to counsel, and Sixth Amendment violation."
    ]
  }
}


In [46]:
response_fine=generate_resp(test_sources[1],model, 'cuda:0')
print(f'ft_model_summary: {response_fine}')

ft_model_summary: ```json
{"Summary_long": "On September 13, 2013, a 49-year-old indigent woman, represented by the Southern Poverty Law Center, filed suit in the Circuit Court of Montgomery County, Alabama, against the city of Montgomery seeking declaratory relief declaring that the jailing of the plaintiff for failure to pay her traffic ticket violated the Fourteenth Amendment\u2019s right to due process and equal protection. Specifically, the plaintiff alleged that her lack of income to pay her traffic ticket fines meant the city had no legitimate justification to incarcerate the plaintiff.\n\nOn November 12, 2013, the case was transferred to the United States District Court for the Middle District of Alabama. The plaintiff filed an amended complaint on December 22, 2013, adding the plaintiff\u2019s right to counsel as a separate cause of action. Shortly thereafter, the plaintiff moved for a temporary restraining order. On March 3, 2014, the defendants argued that a permanent injunc

In [48]:
# First install required packages
!pip install -qU nltk rouge-score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done


In [57]:
pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [60]:
import numpy as np
from bert_score import score as bert_score
from collections import defaultdict

def evaluate_metrics(references, predictions):
    """Unified text evaluation metrics"""
    # Combine reference summaries into single text
    ref_texts = [
        f"Long Summary: {ref['Summary_long']}\nShort Summary: {ref['Summary_short']}"
        for ref in references
    ]
    
    # Convert predictions to single text
    pred_texts = [
        f"Long Summary: {pred.get('Summary_long', '')}\nShort Summary: {pred.get('Summary_short', '')}"
        if isinstance(pred, dict) 
        else str(pred)
        for pred in predictions
    ]

    # Initialize metrics dict
    metrics = defaultdict(dict)

    # BLEU with proper tokenization
    smooth = SmoothingFunction().method4
    metrics['bleu'] = corpus_bleu(
        [[nltk.word_tokenize(ref)] for ref in ref_texts],
        [nltk.word_tokenize(pred) for pred in pred_texts],
        smoothing_function=smooth,
        weights=(0.5, 0.3, 0.2, 0)  # Focus on lower n-grams
    )
    
    # ROUGE
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = defaultdict(list)
    
    for ref, pred in zip(ref_texts, pred_texts):
        scores = scorer.score(ref, pred)
        for key in scores:
            rouge_scores[key].append({
                'f': scores[key].fmeasure,
                'p': scores[key].precision,
                'r': scores[key].recall
            })
    
    # BERTScore
    P, R, F1 = bert_score(pred_texts, ref_texts, lang='en', verbose=False)
    
    metrics.update({
        'rouge': {k: {
            'f': np.mean([v['f'] for v in vals]),
            'p': np.mean([v['p'] for v in vals]),
            'r': np.mean([v['r'] for v in vals])
        } for k, vals in rouge_scores.items()},
        'bert_score': {
            'f1': F1.mean().item(),
            'precision': P.mean().item(),
            'recall': R.mean().item()
        }
    })
    
    return dict(metrics)

def parse_response(response):
    """Unified text extraction from model response"""
    try:
        # Try direct JSON parsing
        json_str = response.split("```json")[-1].split("```")[0].strip()
        data = json_repair.loads(json_str)
        
        return (
            f"Long Summary: {data.get('Summary_long', '')}\n"
            f"Short Summary: {data.get('Summary_short', '')}"
        )
    except:
        # Fallback to text extraction
        if "```json" in response:
            return response.split("```json")[-1].split("```")[0].strip()
        return response.strip()

def evaluate_model(model, tokenizer, test_prompts, references, device, max_samples=None):
    """Unified text evaluation"""
    predictions = []
    model.eval()
    
    test_prompts = test_prompts[:max_samples] if max_samples else test_prompts
    references = references[:max_samples] if max_samples else references
    
    with torch.inference_mode():
        for i, prompt in enumerate(tqdm(test_prompts, desc="Evaluating")):
            try:
                with torch.device(device):
                    response = generate_resp(prompt, model, device)
                    parsed = parse_response(response)
                    predictions.append(parsed)
                    
                if i % 5 == 0:
                    torch.cuda.empty_cache()
                    
            except Exception as e:
                print(f"Error processing sample {i}: {str(e)}")
                predictions.append("")

    return evaluate_metrics(references, predictions)

def print_metrics(name, metrics):
    """Simplified metric printing"""
    print(f"\n{name} Metrics:")
    print(f"BLEU: {metrics['bleu']:.4f}")
    
    print("\nROUGE Scores:")
    for key, vals in metrics['rouge'].items():
        print(f"{key.upper():<8} F1={vals['f']:.4f}  P={vals['p']:.4f}  R={vals['r']:.4f}")
    
    print("\nBERTScore:")
    bert = metrics['bert_score']
    print(f"F1: {bert['f1']:.4f}  Precision: {bert['precision']:.4f}  Recall: {bert['recall']:.4f}")



In [63]:

print("Evaluating Models...")
base_metrics = evaluate_model(base_model, base_tokenizer, test_sources, summarys, 'cuda:1', max_samples=10)
ft_metrics = evaluate_model(model, tokenizer, test_sources, summarys, 'cuda:0', max_samples=10)

print_metrics("Base Model", base_metrics)
print_metrics("Fine-Tuned Model", ft_metrics)

Evaluating Models...


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Error processing sample 3: CUDA out of memory. Tried to allocate 2.31 GiB. GPU 0 has a total capacity of 22.28 GiB of which 1.04 GiB is free. Process 6556 has 21.23 GiB memory in use. Of the allocated memory 17.88 GiB is allocated by PyTorch, and 3.13 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Base Model Metrics:
BLEU: 0.0000

ROUGE Scores:
ROUGE1   F1=0.0136  P=1.0000  R=0.0069
ROUGE2   F1=0.0068  P=0.6667  R=0.0034
ROUGEL   F1=0.0136  P=1.0000  R=0.0069

BERTScore:
F1: 0.7952  Precision: 0.8681  Recall: 0.7338

Fine-Tuned Model Metrics:
BLEU: 0.0979

ROUGE Scores:
ROUGE1   F1=0.3827  P=0.5916  R=0.2977
ROUGE2   F1=0.1740  P=0.2744  R=0.1341
ROUGEL   F1=0.2061  P=0.3211  R=0.1604

BERTScore:
F1: 0.7773  Precision: 0.7849  Recall: 0.7699
